In [2]:
from yelpapi import YelpAPI

import datetime
from dateutil import relativedelta

import numpy as np
import pandas as pd

import itertools

In [3]:
api_key = "X8EDSDndI9422jYtVNksXn629QaCF8exW01ioeybfEEuESu4sFHA99dBMJPMj2SZ6WKj3jA-SV6zVDrkVa8jr9ank1YN3z5Otk5EV4IUBmVW_LkP2XYL8i7pn4MDXHYx"
yelp_api = YelpAPI(api_key)


In [27]:
simple_category_list = [
    "whiskeybars",
    "tikibars",
    "pubs",
    "lounges",
    "beerbar",
    "divebars",
    "speakeasies",
    "bars",
    "cocktailbars",
]

In [110]:
from datetime import datetime
import calendar

d = datetime.now()
d.replace(day=d.day+2,hour=2,minute=30)
open_at = calendar.timegm(d.utctimetuple())

open_at

1570998452

In [111]:
import time

In [112]:
complex_businesses = []

for category in simple_category_list:
    time.sleep(0.2)

    work_businesses = []

    cur_offset = 0

    while True:
        if cur_offset + 50 > 1000: break

        search_results = yelp_api.search_query(
            categories=category,
            sort_by='rating',
            limit=50,
            offset=cur_offset,
            location="566 Guerrero St, San Francisco, CA 94110",
            radius=1000,
            open_at=open_at
        )

        new_businesses = search_results["businesses"]
        if len(new_businesses) == 0: break

        work_businesses.extend(new_businesses)
        cur_offset += 50

    assert len(work_businesses) == np.min([1000,search_results["total"]])
    complex_businesses.extend(work_businesses)

    print(f"{category} - {price+1}, {search_results['total']}")
                  

whiskeybars - 2, 0
tikibars - 2, 0
pubs - 2, 4
lounges - 2, 2
beerbar - 2, 2
divebars - 2, 8
speakeasies - 2, 0
bars - 2, 50
cocktailbars - 2, 11


In [113]:
class Business():

    def __init__(self, name, latitude, longitude, review_count, rating):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.review_count = review_count
        self.rating = rating
        

In [114]:
business_list = [ 
    Business(
        cur_business["name"], *cur_business["coordinates"].values(),
        cur_business["review_count"], cur_business["rating"]
    ) for cur_business in itertools.chain(
        complex_businesses
    ) 
]


In [115]:
business_data = pd.DataFrame.from_records(
    [ cur_business.__dict__ for cur_business in business_list ]
)

business_data.drop_duplicates(inplace=True)

business_data.sort_values(by=["rating", "review_count"], ascending=False, inplace=True)

business_data.reset_index(drop=True, inplace=True)


In [116]:
dd = business_data.copy()
# dd = dd[dd.review_count > 80]
# dd = dd[dd.rating > 4]
sorted(dd.name.tolist())
# np.log10(dd["review_count"]).hist()

["Al's Deli",
 "Alba Ray's",
 'Barrel Proof',
 'Bear Vs Bull',
 'Burma Love',
 'Canela Bistro & Wine Bar',
 'Cava 22',
 'Curio',
 'Dancing Yak',
 'Delirium',
 'El Techo',
 'Elixir',
 'Fig & Thistle Market',
 'Flying Pig Bistro Pub',
 'Foreign Cinema',
 'Gestalt',
 'Giordano Bros',
 'Hi Tops',
 'Il Casaro Pizzeria & Mozzarella Bar - Castro',
 'Last Call Bar',
 'Laszlo',
 'Latin American Club',
 'Little Star Pizza',
 'Lucky 13',
 "Manny's",
 'Mission Bowling Club',
 'Mission Cheese',
 'PRAIRIE',
 'Phoenix Irish Bar',
 'Pilsner Inn',
 'Pizzeria Delfina - Mission',
 'Pork Store Cafe',
 'Skylark',
 'Starbelly',
 'Tacolicious',
 'Teeth',
 'The 500 Club',
 'The Brew Coop',
 'The Crafty Fox Ale House',
 'The Detour',
 'The Liberties',
 "The Monk's Kettle",
 'The Office',
 'The Revolution Cafe',
 'The Sycamore',
 'Thieves Tavern',
 'Uptown',
 'Urban Putt',
 'Woods Cervecería',
 'Zeitgeist']

In [236]:
business_data.to_pickle("./data/pickles/business_data.pkl")
